In [ ]:
!pip install hugsvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 982.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [1]:
from hugsvision.dataio.VisionDataset import VisionDataset

train,test,id2label, label2id = VisionDataset.fromImageFolder(
  "./dataset/",
  test_ratio   = 0.15,
  balanced     = True,
  augmentation = True,
)

ModuleNotFoundError: No module named 'hugsvision'

In [ ]:
huggingface_model = 'google/vit-base-patch16-224-in21k'

In [ ]:
from hugsvision.nnet.VisionClassifierTrainer import VisionClassifierTrainer
from transformers import ViTFeatureExtractor, ViTForImageClassification

trainer = VisionClassifierTrainer(
	model_name   = "MyKvasirV2Model",
	train        = train,
	test         = test,
	output_dir   = "./out/",
	max_epochs   = 10,
	batch_size   = 32,
	lr	     = 2e-5,
	fp16	     = True,
	model = ViTForImageClassification.from_pretrained(
	    huggingface_model,
	    num_labels = len(label2id),
	    label2id   = label2id,
	    id2label   = id2label
	),
	feature_extractor = ViTFeatureExtractor.from_pretrained(
		huggingface_model,
	),
)

In [ ]:
ref, hyp = trainer.evaluate_f1_score()

100%|██████████| 270/270 [00:21<00:00, 12.52it/s]


              precision    recall  f1-score   support

  earthquake     0.9890    0.9783    0.9836        92
       flood     0.9778    0.9888    0.9832        89
    wildfire     1.0000    1.0000    1.0000        89

    accuracy                         0.9889       270
   macro avg     0.9889    0.9890    0.9889       270
weighted avg     0.9889    0.9889    0.9889       270

Logs saved at: ./out/MYKVASIRV2MODEL/10_2024-03-22-00-08-07


In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ref, hyp)
labels = list(label2id.keys())
df_cm = pd.DataFrame(cm, index = labels, columns = labels)

plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True, annot_kws={"size": 8}, fmt="")
plt.savefig("./imgs/conf_matrix_1.jpg")

In [ ]:
import os.path
from transformers import ViTFeatureExtractor, ViTForImageClassification
from hugsvision.inference.VisionClassifierInference import VisionClassifierInference

path1 = "./out/MYKVASIRV2MODEL/10_2024-03-22-00-08-07/feature_extractor/"
path2 = "./out/MYKVASIRV2MODEL/10_2024-03-22-00-08-07/model/"
img  = "C:/Users/rohan/OneDrive/Desktop/DisasterManage/Disaster-Management/Disasterpics/china-earthquake-21.jpg"

classifier = VisionClassifierInference(
    feature_extractor = ViTFeatureExtractor.from_pretrained(path1),
    model = ViTForImageClassification.from_pretrained(path2),
)

label = classifier.predict(img_path=img)
print("Predicted class:", label)

c:\Users\rohan\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Model loaded!
Predicted class: earthquake
